# **Sound Spot**

##### Installation et importations

In [1]:
# %pip install spotipy
# %pip install selenium
# %pip install pandas
# %pip install --upgrade pip
# %pip install bcrypt
# %pip install mysql
# %pip install mysql-connector-python
# %pip install librosa
# TODO: LIBROSA!!!
import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
# import json

import requests

from selenium import webdriver

import sys
sys.path.append('GitHub\\data-camp\\secured_passwd.py')
from secured_passwd import CLIENT_ID, CLIENT_SECRET, SQL_PASSWD, hash_ID, hash_SECRET, hash_SQL

Authorization Site : https://accounts.spotify.com/fr/authorize?client_id=ee862f2c84fe4f00929c71fb2eb50807&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com&scope=playlist-read-private

In [2]:
# IDs
client_id = hash_ID
client_secret = hash_SECRET

##### Accéder aux infos Spotify de l'utilisateurs pour créer notre BDD

In [3]:
# Créez une instance de Spotipy avec des identifiants client
sp_oauth = SpotifyOAuth(
    client_id = client_id,
    client_secret = client_secret,
    redirect_uri = 'https://open.spotify.com',
    scope = 'user-library-read'  # Autorisations nécessaires, y compris la lecture des playlists privées
)


In [4]:
# util.prompt_for_user_token(username,scope,client_id='your-spotify-client-id',client_secret='your-spotify-client-secret',redirect_uri='your-app-redirect-url')

# Obtenir le jeton d'accès ou le renouveler s'il est expiré
token_info = sp_oauth.get_cached_token()
print("TOKEN EXISTS : ", token_info)

# Vérifiez si le jeton d'accès est valide
if not token_info:

    auth_url = sp_oauth.get_authorize_url()

    driver = webdriver.Chrome()

    driver.get(auth_url)

    # Attendez que l'utilisateur autorise l'application et soit redirigé
    input("Appuyez sur Entrée une fois redirigé vers l'URL de redirection...")

    final_url = driver.current_url

    driver.quit()
    
    response = requests.get(final_url)
    print('RESPONSE : ', response.url)
    code = sp_oauth.get_authorization_code(response.url)
    print("CODE : ", code)

    access_token = sp_oauth.get_access_token(code, as_dict = False)
    print("TOKEN : ", access_token)

else:

    access_token = token_info['access_token']

TOKEN EXISTS :  {'access_token': 'BQDdjLnMxVXaMgSZYwNhQa_xRfrRDeTG8ctiqU78zCrYNDbuXrIXH47r8jMREdEhzS3UQmqyG2dMI_Y72D7kg_O-asLPhz3oCgBWjDAdsuyyZcDYwcJSXPGw9ehJlFn6E4Rnx-B3wqvlcZeUKtoIKxwiQWVD7cO3EXOAxQgyCGFNYM0_EVt2eiMC2DUCPM8oiXSJZk2aJhJPR90', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read', 'expires_at': 1698513526, 'refresh_token': 'AQAB3IwpoalkzrKScKRNhoR76RKMaJdsbysdJxb4-QWmVvsj0RFSDguZwJumJbwC8QOh24FQj0LoIfIo2xdC0ooc2p3vn6i7w6ttDa37pO6VMh8JfDCE9hJuVST1Hwyi1rg'}


In [5]:
# S'authentifier avec le jeton d'accès
sp = spotipy.Spotify(auth=access_token)

# ID de la playlist Spotify que vous souhaitez récupérer (vous pouvez obtenir l'ID depuis l'URL de la playlist)
playlist_id = 'https://open.spotify.com/playlist/4QnaHIvkm489VZOp4CWeTB?si=e11e9a34315143db&pt=3c156a8c357198ee10d0f85f529b02b3' # Private Playlist
# playlist_id = 'https://open.spotify.com/playlist/37i9dQZEVXbhGLvgw0jIZm?si=d1e5dfd1966c45b8' # Public Playlist

# Récupérez les informations sur la playlist
playlist = sp.playlist(playlist_id)

# Imprimez les informations sur la playlist
# print(f"Nom de la playlist : {playlist['name']}")
# print(f"Propriétaire de la playlist : {playlist['owner']['display_name']}")
# print(f"Nombre de pistes dans la playlist : {playlist['tracks']['total']}")

# Liste des noms des pistes (en paginant)
track_data = []
results = sp.playlist_tracks(playlist_id)

column_names = [
    'Track ID', 'Main Artist ID', 'Track Name', 'Track Popularity',
    'Track Released Date', 'Main Artist', 'Main Artist Genre',
    'Main Artist Popularity', 'Other Artists', 'Track URL',
    'Track Image URL', 'Main Artist Image URL'
]
# data = pd.DataFrame()

while results:
    for track in results['items']:
        # Track info
        track_info = track['track']

        # Artists info
        artists = track['track']['artists']
        other_artists = []

        main_artist_id = track_info['artists'][0]['id']
        main_artist_info = sp.artist(main_artist_id)

        for artist in artists:
            other_artists.append(artist['name'])
        
        other_artists.pop(0)

        if 'images' in main_artist_info and main_artist_info['images']:
            main_artist_image_url = main_artist_info['images'][0]['url']
        elif 'image' in main_artist_info:
            main_artist_image_url = main_artist_info['image']['url']
        else:
            main_artist_image_url = None

        track_data.append([
            track_info['id'],
            main_artist_id,
            track_info['name'],
            track_info['popularity'],
            track_info['album']['release_date'],
            main_artist_info['name'],
            main_artist_info['genres'],
            main_artist_info['popularity'],
            other_artists,
            track_info['external_urls'],
            track_info['album']['images'][0]['url'],
            main_artist_image_url
        ])
        
    results = sp.next(results)  # Chargez la page suivante

for idx in track_data:

    data = pd.DataFrame(track_data, columns=column_names)

    # The new version deprecating and don't accept .append anymore :(

    # data = data.append(
    #     {
    #         'Track ID' : song_info[0],
    #         'Main Artist ID' : song_info[1],
    #         'Track Name': song_info[2],
    #         'Track Popularity': song_info[3],
    #         'Track Released Date' : song_info[4],
    #         'Main Artist': song_info[5],
    #         'Main Artist Genre': song_info[6],
    #         'Main Artist Popularity': song_info[7],
    #         'Other Artists': song_info[8],
    #         'Track URL' : song_info[9],
    #         'Track Image URL' : song_info[10],
    #         'Main Artist Image URL' : song_info[11]
    #     },
    #     ignore_index=True
    # )

In [6]:
data.head(5)

,Track ID,Main Artist ID,Track Name,Track Popularity,Track Released Date,Main Artist,Main Artist Genre,Main Artist Popularity,Other Artists,Track URL,Track Image URL,Main Artist Image URL
0,2UN0lp72LAusrXi8LLVomt,6eUKZXaKkcviH0Ku9w2n3V,2step (feat. Lil Baby),68,2022-04-22,Ed Sheeran,"[pop, singer-songwriter pop, uk pop]",88,[Lil Baby],{'spotify': 'https://open.spotify.com/track/2U...,https://i.scdn.co/image/ab67616d0000b273fa1f47...,https://i.scdn.co/image/ab6761610000e5eb3bcef8...
1,357EwLZfD9vuGTZ7dxqumw,7wlFDEWiM5OoIAt8RSli8b,Akbar,56,2018-06-15,YoungBoy Never Broke Again,"[baton rouge rap, rap]",81,[],{'spotify': 'https://open.spotify.com/track/35...,https://i.scdn.co/image/ab67616d0000b273b233d2...,https://i.scdn.co/image/ab6761610000e5eba213bd...
2,32vE1nuG8T9c8bhmZdRY6d,246dkjvS1zLTtiykXe5h60,Cooped Up (with Roddy Ricch),73,2022-06-03,Post Malone,"[dfw rap, melodic rap, pop, rap]",88,[Roddy Ricch],{'spotify': 'https://open.spotify.com/track/32...,https://i.scdn.co/image/ab67616d0000b273343626...,https://i.scdn.co/image/ab6761610000e5eb6be070...
3,4ZpIuzx91EAPK3VimONbfB,1rpxM2mBJnWdfPeieLDYOc,FADE UP,70,2022-06-24,ZEG P,[],49,"[Hamza, SCH]",{'spotify': 'https://open.spotify.com/track/4Z...,https://i.scdn.co/image/ab67616d0000b2730568d3...,https://i.scdn.co/image/ab6761610000e5eb0199da...
4,27NovPIUIRrOZoCHxABJwK,7jVv8c5Fj3E9VhNjxT4snq,INDUSTRY BABY (feat. Jack Harlow),81,2021-07-23,Lil Nas X,[lgbtq+ hip hop],76,[Jack Harlow],{'spotify': 'https://open.spotify.com/track/27...,https://i.scdn.co/image/ab67616d0000b273ba2667...,https://i.scdn.co/image/ab6761610000e5ebd66f1e...


##### **Retrouver les infos d'une chanson juste en tapant le nom de la chanson et celui de l'artiste**

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configurez l'authentification
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Remplacez ces valeurs par le nom de la chanson et de l'artiste que vous recherchez
chanson = 'The Hills'
artiste = 'The Weeknd'

# Recherchez la chanson
results = sp.search(q=f'track:{chanson} artist:{artiste}', type='track')

# Obtenez les données de la première chanson de la liste de résultats
if len(results['tracks']['items']) > 0:
    chanson_data = results['tracks']['items'][0]
    print('Nom de la chanson:', chanson_data['name'])
    print('Artiste:', chanson_data['artists'][0]['name'])
    print('Album:', chanson_data['album']['name'])
    print('URL Spotify:', chanson_data['external_urls']['spotify'])
else:
    print('Chanson non trouvée.')


Nom de la chanson: The Hills
Artiste: The Weeknd
Album: The Highlights
URL Spotify: https://open.spotify.com/track/33kxP9sQOzB3fRL8zl17uK


##### MySQL Connection

In [15]:
import mysql.connector

# Create a connexion to the database
connexion = mysql.connector.connect(
    host='localhost',
    user='soundspothandler',
    password=SQL_PASSWD,
    database='sound_spot'
)

# Create a cursor to execute SQL queries
cursor = connexion.cursor()

# Exécutez une requête SQL (par exemple, sélectionnez toutes les chansons de votre base de données)
query = "SELECT * FROM TRACK"
cursor.execute(query)

# Récupérez les résultats de la requête
resultats = cursor.fetchall()

# Parcourez les résultats
for resultat in resultats:
    print(resultat)

In [ ]:
# Nom de l'album, de l'artiste
# Type de chanson, rythmique, popularité
# Lyrics --> Webscrapping

# À voir : category, category_playlists, featured_playlists
# playlist_add_items, playlist_change_details, artist_albums
# artist_related_artists, artist_top_tracks, audio_analysis,
# audio_features, playlist_cover_image, playlist_items
# recommendation_genre_seeds, recommandations, search,
# seek_track, user_playlists, get_access_token...

# Possibilité d'écouter de l'audio depuis le site

# Filtre pour ranger dans un certain ordre



# ................................................................
# # Retrieve the user's top tracks
# top_tracks = sp.current_user_top_tracks(limit=10, time_range='medium_term')  # You can adjust the limit and time_range

# # Print the top tracks
# for idx, track in enumerate(top_tracks['items'], start=1):
#     print(f"{idx}. {track['name']} by {', '.join([artist['name'] for artist in track['artists']])}")

In [ ]:
# sp.artists('https://open.spotify.com/intl-fr/track/7IBSffWIu7P2MC7kMwy2FM?si=b0c36190df6d467b')

In [ ]:
# track['track']['artists']['id']

In [16]:
# results = sp.playlist_tracks(playlist_id)
# print(results)

HTTP Error for GET to https://api.spotify.com/v1/playlists/4QnaHIvkm489VZOp4CWeTB/tracks with Params: {'limit': 100, 'offset': 0, 'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Not found.


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/playlists/4QnaHIvkm489VZOp4CWeTB/tracks?limit=100&offset=0&additional_types=track:
 Not found., reason: None

In [ ]:
# print(playlist)

In [ ]:
# import spotipy
# from spotipy.oauth2 import SpotifyOAuth

# scope = "user-library-read"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# results = sp.current_user_saved_tracks()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])